In [1]:
import numpy as np
import pandas as pd
import math
from sklearn.metrics import mean_squared_error

In [2]:
data=pd.read_csv("/home/hithesh/Downloads/creditcard.csv")
print(data.shape)
data.head()

(284807, 31)


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
import seaborn as sns
corr = data.corr()
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);
print(corr.iloc[30,:29]) #To find the features which are most correlated to Class column

Time   -0.012323
V1     -0.101347
V2      0.091289
V3     -0.192961
V4      0.133447
V5     -0.094974
V6     -0.043643
V7     -0.187257
V8      0.019875
V9     -0.097733
V10    -0.216883
V11     0.154876
V12    -0.260593
V13    -0.004570
V14    -0.302544
V15    -0.004223
V16    -0.196539
V17    -0.326481
V18    -0.111485
V19     0.034783
V20     0.020090
V21     0.040413
V22     0.000805
V23    -0.002685
V24    -0.007221
V25     0.003308
V26     0.004455
V27     0.017580
V28     0.009536
Name: Class, dtype: float64


In [4]:
features=['V3','V7','V10','V11','V12','V14','V16','V17','Class']
# features=['V1','V2','V3','V4','V5','V7','V9','V10','V11','V12','V14','V16','V17','V18','Class']
train_features=['V3','V7','V10','V11','V12','V14','V16','V17']
# train_features=['V1','V2','V3','V4','V5','V7','V9','V10','V11','V12','V14','V16','V17','V18']

In [5]:
ind1=[i for i in range(data.shape[0]) if data.loc[i,'Class']==1]
ind0=[i for i in range(data.shape[0]) if data.loc[i,'Class']==0]
data1=data.loc[ind1,features]
data0=data.loc[ind0,features]

In [6]:
#Training,CV,Test Split 
n1=int(0.6*len(ind0))
n2=int(0.2*len(ind0))
n3=int(0.5*len(ind1))

train_data = data0.loc[ind0[:n1],train_features]
cv_data = pd.concat((data0.loc[ind0[n1:n1+n2],features],data1.loc[ind1[:n3],features]),axis=0)
test_data = pd.concat((data0.loc[ind0[n1+n2:],features],data1.loc[ind1[n3:],features]),axis=0)
print(cv_data.shape)

(57109, 9)


In [7]:
def prob(x,mu,ss):
    return math.exp(-(x-mu)**2/(2*ss))/(math.sqrt(2*math.pi*ss))
print(prob(0,0,1),1/(math.sqrt(2*math.pi)))

0.3989422804014327 0.3989422804014327


In [8]:
#Training the model
mu=[]
ss=[]
for i in range(train_data.shape[1]):
    mu.append(sum(train_data.iloc[:,i])/train_data.shape[0])
    diff=train_data.iloc[:,i]-mu[i]*pd.Series(np.ones(train_data.shape[0]))
#     print(train_data.iloc[:,i],mu[i]*pd.Series(np.ones(train_data.shape[0])))
    err=mean_squared_error(train_data.iloc[:,i],mu[i]*pd.Series(np.ones(train_data.shape[0])))
    ss.append(err)

In [24]:
#Cross Validation
pred=np.ones(cv_data.shape[0])
epsilon=10**-50
for i in range(cv_data.shape[0]):
    p=1
    for j in range(cv_data.shape[1]-1):
        p = p * prob(cv_data.iloc[i,j],mu[j],ss[j])
    if(p<epsilon):
        pred[i]=1
    else:
        pred[i]=0
print(pred.mean())

0.0033970127300425503


In [25]:
pred=pd.Series(pred)
tp=0
fp=0
tn=0
fn=0
for i in range(cv_data.shape[0]):
    if(pred[i]==cv_data.iloc[i,-1] and pred[i]==1):
        tp=tp+1
    elif(pred[i]==cv_data.iloc[i,-1] and pred[i]==0):
        tn=tn+1
    elif(pred[i]!=cv_data.iloc[i,-1] and pred[i]==1):
        fp=fp+1
    else:
        fn=fn+1
print(tp,fp)
print(fn,tn)
P=tp/(tp+fp)
R=tp/(tp+fn) 
f1_score=2*P*R/(P+R)
print(P,R,f1_score)

148 46
98 56817
0.7628865979381443 0.6016260162601627 0.6727272727272727


In [26]:
#Test
pred=np.ones(test_data.shape[0])
epsilon=10**-50
for i in range(test_data.shape[0]):
    p=1
    for j in range(test_data.shape[1]-1):
        p = p * prob(test_data.iloc[i,j],mu[j],ss[j])
    if(p<epsilon):
        pred[i]=1
    else:
        pred[i]=0

In [52]:
right=0
flagged=0
wrong=0
for i in range(test_data.shape[0]):
    if(pred[i]==test_data.iloc[i,-1]):
        right = right + 1
    if(pred[i]==test_data.iloc[i,-1] and test_data.iloc[i,-1]==1):
        flagged = flagged+1
    if(pred[i]!=test_data.iloc[i,-1] and test_data.iloc[i,-1]==0):
        wrong = wrong + 1
accuracy = right/len(pred)

print('Total transactions: ' + str(n2+n3))

print(str(flagged)+' fraud transactions flagged out of ' + str(n3)+ ' fraud transactions made')

print(str(wrong)+' non-fraud transactions flagged fraud.')

print('Accuracy of prediction: '+str(accuracy))

Total transactions: 57109
124 fraud transactions flagged out of 246 fraud transactions made
41 non-fraud transactions flagged fraud.
Accuracy of prediction: 0.9971458088917684
